In [1]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"C:\Users\shaif\Downloads\Compressed\che\chest_xray\train"

# Initialize empty lists for images and labels
X_train = []
Y_train = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpeg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_train.append(img_array)
            Y_train.append(class_label)
            
    class_label = class_label + 1

    
# Convert lists to numpy arrays
X_train = np.array(X_train)
Y_train = np.array(Y_train)

print(f'X_train shape: {X_train.shape}')
print(f'Y_train shape: {Y_train.shape}')

100%|█████████████████████████████████████████████████████████████████████| 2/2 [01:55<00:00, 57.73s/it]

X_train shape: (5216, 32, 32, 3)
Y_train shape: (5216,)


In [2]:
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tqdm import tqdm

# Define the path to the directory
directory_path = r"C:\Users\shaif\Downloads\Compressed\che\chest_xray\val"

# Initialize empty lists for images and labels
X_val = []
Y_val = []
class_label = 0

# Loop through subdirectories (classes)
for class_folder in tqdm(sorted(os.listdir(directory_path))):
    class_path = os.path.join(directory_path, class_folder)
    
    for image_file in os.listdir(class_path):
        if image_file.endswith('.jpeg'):
            image_path = os.path.join(class_path, image_file)
            
            # Load image, convert to RGB and resize
            img = Image.open(image_path).convert('RGB')
            img = img.resize((32, 32))
            img_array = np.array(img)
            
            # Append image and label to lists
            X_val.append(img_array)
            Y_val.append(class_label)
            
    class_label = class_label + 1

# Convert lists to numpy arrays
X_val = np.array(X_val)
Y_val = np.array(Y_val)

print(f'X_train shape: {X_val.shape}')
print(f'Y_train shape: {Y_val.shape}')

100%|█████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00,  5.50it/s]

X_train shape: (16, 32, 32, 3)
Y_train shape: (16,)


In [3]:
import tensorflow as tf
#tf.keras.backend.clear_session()
from tensorflow.keras.utils import to_categorical
YT = to_categorical(Y_train)
VT = to_categorical(Y_val)

In [4]:
Y_val

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1])

In [4]:
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 2
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(32, 32, 3))    
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

Epoch 1/50
147/147 [==============================] - 23s 59ms/step - loss: 0.2710 - accuracy: 0.9256 - val_loss: 0.9290 - val_accuracy: 0.8448
Epoch 2/50
147/147 [==============================] - 6s 43ms/step - loss: 0.0979 - accuracy: 0.9672 - val_loss: 0.1380 - val_accuracy: 0.9559
Epoch 3/50
147/147 [==============================] - 7s 47ms/step - loss: 0.0782 - accuracy: 0.9729 - val_loss: 0.1661 - val_accuracy: 0.9617
Epoch 4/50
147/147 [==============================] - 7s 47ms/step - loss: 0.0710 - accuracy: 0.9778 - val_loss: 0.0663 - val_accuracy: 0.9828
Epoch 5/50
147/147 [==============================] - 8s 53ms/step - loss: 0.0605 - accuracy: 0.9800 - val_loss: 0.0740 - val_accuracy: 0.9751
Epoch 6/50
147/147 [==============================] - 7s 51ms/step - loss: 0.0379 - accuracy: 0.9855 - val_loss: 0.0946 - val_accuracy: 0.9770
Epoch 7/50
147/147 [==============================] - 7s 50ms/step - loss: 0.0300 - accuracy: 0.9904 - val_loss: 0.0267 - val_accuracy: 0.994

In [15]:
model.fit(X_train,YT, epochs=2,batch_size=32, validation_split = .1)

Epoch 1/2
147/147 [==============================] - 5s 31ms/step - loss: 0.0252 - accuracy: 0.9906 - val_loss: 0.1588 - val_accuracy: 0.9617
Epoch 2/2
147/147 [==============================] - 4s 28ms/step - loss: 0.0447 - accuracy: 0.9834 - val_loss: 0.0130 - val_accuracy: 0.9923


In [5]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix

# Make predictions on the test dataset
predictions = model.predict(X_val)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9375
F1 Score: 0.9372549019607843


In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [6]:
import tensorflow
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
num_classes = 2
model = tensorflow.keras.models.load_model(r"D:\che_model.h5")

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D(keepdims=False)(x)  # Explicitly specify keepdims
output = Dense(num_classes, activation='softmax')(x)
new_model = tensorflow.keras.models.Model(inputs=model.input, outputs=output) 
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
epochs = 50
new_model.fit(X_train,YT, epochs=epochs,batch_size=32, validation_split = .1)

Epoch 1/50


C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizer_v2\adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


147/147 [==============================] - 13s 49ms/step - loss: 0.2205 - accuracy: 0.9372 - val_loss: 0.3206 - val_accuracy: 0.9349
Epoch 2/50
147/147 [==============================] - 7s 51ms/step - loss: 0.0884 - accuracy: 0.9683 - val_loss: 0.0416 - val_accuracy: 0.9847
Epoch 3/50
147/147 [==============================] - 7s 49ms/step - loss: 0.0560 - accuracy: 0.9757 - val_loss: 0.1471 - val_accuracy: 0.9579
Epoch 4/50
147/147 [==============================] - 8s 53ms/step - loss: 0.1027 - accuracy: 0.9766 - val_loss: 0.0118 - val_accuracy: 0.9943
Epoch 5/50
147/147 [==============================] - 8s 51ms/step - loss: 0.1038 - accuracy: 0.9644 - val_loss: 0.1877 - val_accuracy: 0.9598
Epoch 6/50
147/147 [==============================] - 7s 49ms/step - loss: 0.0539 - accuracy: 0.9821 - val_loss: 0.1857 - val_accuracy: 0.9559
Epoch 7/50
147/147 [==============================] - 6s 44ms/step - loss: 0.0324 - accuracy: 0.9889 - val_loss: 0.0977 - val_accuracy: 0.9847
Epoch 8/5

In [9]:
new_model.fit(X_train,YT, epochs=2,batch_size=32, validation_split = .1)

Epoch 1/2
147/147 [==============================] - 7s 48ms/step - loss: 1.8576e-04 - accuracy: 1.0000 - val_loss: 0.1706 - val_accuracy: 0.9828
Epoch 2/2
147/147 [==============================] - 6s 44ms/step - loss: 8.9072e-05 - accuracy: 1.0000 - val_loss: 0.1774 - val_accuracy: 0.9808


In [10]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
# Make predictions on the test dataset
predictions = new_model.predict(X_val)
# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)
# Calculate accuracy
accuracy = accuracy_score(Y_val, predicted_labels)
# Calculate F1 score
f1 = f1_score(Y_val, predicted_labels, average='weighted')
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")

Accuracy: 0.9375
F1 Score: 0.9372549019607843


In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [8]:
from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
    
num_classes = 10  
model = keras.models.load_model(r"C:\Users\shaif\Downloads\imb_ssl.h5")
x = model.layers[-8].output  
output = Dense(num_classes, activation='softmax')(x)
new_model = keras.models.Model(inputs=model.input, outputs=output)
new_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])
new_model.fit(X_train, YT, batch_size=64, epochs=50, validation_split = .2)

Epoch 1/50
363/363 [==============================] - 42s 68ms/step - loss: 516.2614 - accuracy: 0.1743 - val_loss: 4347.4189 - val_accuracy: 0.0000e+00
Epoch 2/50
363/363 [==============================] - 21s 58ms/step - loss: 193.0619 - accuracy: 0.1802 - val_loss: 2157.3953 - val_accuracy: 0.0109
Epoch 3/50
363/363 [==============================] - 22s 61ms/step - loss: 62.5039 - accuracy: 0.1754 - val_loss: 1404.2903 - val_accuracy: 0.1643
Epoch 4/50
363/363 [==============================] - 23s 63ms/step - loss: 34.9450 - accuracy: 0.1793 - val_loss: 934.8641 - val_accuracy: 0.0000e+00
Epoch 5/50
363/363 [==============================] - 20s 56ms/step - loss: 20.3900 - accuracy: 0.1827 - val_loss: 715.6703 - val_accuracy: 0.0000e+00
Epoch 6/50
363/363 [==============================] - 22s 61ms/step - loss: 15.4835 - accuracy: 0.1938 - val_loss: 586.2610 - val_accuracy: 0.0000e+00
Epoch 7/50
363/363 [==============================] - 23s 65ms/step - loss: 12.3891 - accuracy: 0

In [9]:
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

# Assuming you have trained a model and obtained predicted probabilities on x_test
y_pred_prob = new_model.predict(V_val)

# Convert predicted probabilities to predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Convert y_test to predicted labels format
y_test_labels = np.argmax(y_val, axis=1)

# Calculate accuracy
accuracy = accuracy_score(y_test_labels, y_pred)
print("Accuracy:", accuracy)

# Calculate F1 score (micro-average)
f1_micro = f1_score(y_test_labels, y_pred, average='micro')
print("F1 Score (Micro):", f1_micro)

NameError: name 'V_val' is not defined

In [ ]:
print(".......................................................................................")
print(".......................................................................................")

In [ ]:
print(".......................................................................................")
print(".......................................................................................")